In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
import numpy as np

# Load MNIST dataset and preprocess
from keras.utils import to_categorical
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


train_images, test_images = train_images / 255.0, test_images / 255.0


# One-hot encode labels
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

# Reshape images to (28, 28, 1) for Conv2D layer
train_images_reshaped = train_images.reshape((-1, 28, 28, 1))
test_images_reshaped = test_images.reshape((-1, 28, 28, 1))

In [4]:
gen = ImageDataGenerator(rotation_range = 12, width_shift_range = 0.1, shear_range = 0.1,
                         height_shift_range = 0.1, zoom_range = 0.1, fill_mode = 'nearest', horizontal_flip = False,
                         vertical_flip = False, featurewise_center = False,
                         samplewise_center = False, featurewise_std_normalization = False,
                         samplewise_std_normalization = False)
test_gen = ImageDataGenerator()

# Create batches to  train models faster
train_generator = gen.flow(train_images_reshaped,train_labels, batch_size = 32)
test_generator = test_gen.flow(test_images_reshaped, test_labels, batch_size = 32)

In [5]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3, verbose = 1, factor = 0.4, min_lr = 0.00002,
                                            mode = 'auto', cooldown = 0)


In [6]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras import models
from keras.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation, BatchNormalization

# Function to create and train the specified CNN model
def create_and_train_model():
    
        model = Sequential()

        model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                        input_shape=(28,28,1)))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
        model.add(Dropout(0.25))

        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))


    # Compile the model
        model.compile(optimizer=Adam(learning_rate=0.01), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

#     # Reshape images to (28, 28, 1) for Conv2D layer
#      train_images_reshaped = train_images.reshape((-1, 28, 28, 1))
#      test_images_reshaped = test_images.reshape((-1, 28, 28, 1))

        history = model.fit_generator(train_generator, steps_per_epoch = 40000//16, epochs = 10, 
                              validation_data = test_generator, validation_steps = 10000//8, verbose = 1,
                              callbacks=[reduce_lr])

        # evaluate the model
        scores = model.evaluate(test_images_reshaped, test_labels, verbose = 2)

        return scores[1]

num_models = 5
test_accuracies = []

for i in range(num_models):
    acurracy = create_and_train_model()
    test_accuracies.append(acurracy)
    print(f'Model-{i+1} test accuracy: {acurracy:.4f}')

# Compute the mean test accuracy
mean_test_accuracy = np.mean(test_accuracies)
print(f'Mean Test Accuracy across Models: {mean_test_accuracy}')

Epoch 1/10


C:\Users\ashuo\AppData\Local\Temp\ipykernel_33420\1689458331.py:46: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch = 40000//16, epochs = 10,


2500/2500 [==============================] - 150s 60ms/step - loss: 0.4856 - accuracy: 0.8877 - val_loss: 4.1094 - val_accuracy: 0.6338 - lr: 0.0100
313/313 - 5s - loss: 4.1094 - accuracy: 0.6338 - 5s/epoch - 17ms/step
Model-1 test accuracy: 0.6338
Epoch 1/10
2500/2500 [==============================] - 162s 64ms/step - loss: 0.4783 - accuracy: 0.8911 - val_loss: 0.1084 - val_accuracy: 0.9683 - lr: 0.0100
313/313 - 6s - loss: 0.1084 - accuracy: 0.9683 - 6s/epoch - 20ms/step
Model-2 test accuracy: 0.9683
Epoch 1/10
2500/2500 [==============================] - 151s 60ms/step - loss: 0.4490 - accuracy: 0.8995 - val_loss: 0.0590 - val_accuracy: 0.9834 - lr: 0.0100
313/313 - 5s - loss: 0.0590 - accuracy: 0.9834 - 5s/epoch - 18ms/step
Model-3 test accuracy: 0.9834
Epoch 1/10
2500/2500 [==============================] - 150s 60ms/step - loss: 0.4471 - accuracy: 0.9014 - val_loss: 0.0560 - val_accuracy: 0.9825 - lr: 0.0100
313/313 - 5s - loss: 0.0560 - accuracy: 0.9825 - 5s/epoch - 17ms/step
M